<a href="https://colab.research.google.com/github/anubha42/Mystuff/blob/main/transformer_self_with_transformer_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRANSFORMER BLOCK INCLUDED BUT NO POSITIONAL ENCODING

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import matplotlib.pyplot as plt

In [ ]:
seq_length=4 #lenght of input sentence
input_dim=512 #input
d_model=512 #op of attention for every word
batch_size=1 #parallel processing

In [ ]:
x=torch.rand((batch_size, seq_length, input_dim )) #given to multihead attention no position encoding
print(x)
print(x.shape)

tensor([[[0.6116, 0.7832, 0.9442,  ..., 0.4138, 0.5190, 0.9550],
         [0.9130, 0.4924, 0.9587,  ..., 0.5033, 0.6682, 0.1424],
         [0.9066, 0.7734, 0.5284,  ..., 0.9671, 0.2104, 0.5681],
         [0.7740, 0.6338, 0.5152,  ..., 0.1152, 0.7930, 0.8525]]])
torch.Size([1, 4, 512])


In [ ]:
qkv_layer=nn.Linear(input_dim, 3*d_model) #all concatenated 3 layers
qkv=qkv_layer(x)
qkv.shape  #1 batch 4 words of (3x512=1536 size)

torch.Size([1, 4, 1536])

In [ ]:
num_heads=8
head_dim = d_model // num_heads #512/8=64
qkv=qkv.reshape(batch_size, seq_length, num_heads, 3*head_dim) # 8x192= 1536
print(qkv.shape)

torch.Size([1, 4, 8, 192])


In [ ]:
qkv=qkv.permute(0,2,1,3) #(batch_Size, num_heads, seq_lenght, 3*head_dim) easy for computation
qkv.shape

torch.Size([1, 8, 4, 192])

In [ ]:
q,k,v=qkv.chunk(3, dim=-1) #splitting into 3 along the last dimension
q.shape,k.shape ,v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [ ]:
import math
d_k=q.size()[-1]
scaled=torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)
scaled.shape

torch.Size([1, 8, 4, 4])

In [ ]:
mask=torch.full(scaled.size(), float('-inf'))
print(mask.shape)
print(mask[0][1])
mask=torch.triu(mask, diagonal=1) #upper triangular
mask[0][1]

torch.Size([1, 8, 4, 4])
tensor([[-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf]])


tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [ ]:
(scaled+mask)[0][1] #mask for decoder

tensor([[ 0.0256,    -inf,    -inf,    -inf],
        [-0.0351, -0.0300,    -inf,    -inf],
        [-0.0325, -0.0903, -0.0160,    -inf],
        [-0.0309, -0.0584,  0.0031, -0.0696]], grad_fn=<SelectBackward0>)

In [ ]:
def scaled_dot_product(q,k,v,mask=None):
  d_k=q.size()[-1]
  scaled=torch.matmul(q,k.transpose(-1,-2)) / math.sqrt(d_k)
  if mask is not None:
    scaled+=mask
  attention=f.softmax(scaled,dim=-1)
  value=torch.matmul(attention, v)
  return value, attention

In [ ]:
value, attention = scaled_dot_product(q,k,v, mask=None) #encoder
attention.shape

torch.Size([1, 8, 4, 4])

In [ ]:
value.shape #values after multiplication with attention

torch.Size([1, 8, 4, 64])

In [ ]:
attention[0][0] #encoder attention

tensor([[0.2484, 0.2585, 0.2361, 0.2569],
        [0.2405, 0.2487, 0.2535, 0.2573],
        [0.2399, 0.2574, 0.2399, 0.2629],
        [0.2377, 0.2555, 0.2448, 0.2620]], grad_fn=<SelectBackward0>)

In [ ]:
value, attention = scaled_dot_product(q,k,v, mask=mask) #decoder attention
attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4915, 0.5085, 0.0000, 0.0000],
        [0.3254, 0.3492, 0.3254, 0.0000],
        [0.2377, 0.2555, 0.2448, 0.2620]], grad_fn=<SelectBackward0>)

In [ ]:
#concatenating values
value=value.reshape(batch_size, seq_length, num_heads*head_dim)
value.shape

torch.Size([1, 4, 512])

In [ ]:
q1=q.reshape(batch_size, seq_length, num_heads*head_dim)
k1=k.reshape(batch_size, seq_length, num_heads*head_dim)
v1=v.reshape(batch_size, seq_length, num_heads*head_dim)
q1.shape

torch.Size([1, 4, 512])

In [ ]:
multihead_attn = nn.MultiheadAttention(d_model, num_heads) #using pytorch existing module
attn_output, attn_output_weights = multihead_attn(q1,k1,v1)
print(attn_output.shape)

torch.Size([1, 4, 512])


In [ ]:
attn_output_weights.shape
attn_output

tensor([[[-0.3938,  0.1054,  0.2816,  ...,  0.1038,  0.0654,  0.2489],
         [ 0.1186, -0.0082, -0.1519,  ..., -0.0367, -0.0336,  0.1187],
         [-0.1626,  0.1014, -0.0364,  ..., -0.1913,  0.0073, -0.1830],
         [ 0.2504, -0.1506, -0.0716,  ..., -0.2099, -0.1274,  0.0694]]],
       grad_fn=<ViewBackward0>)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

In [ ]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [ ]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, hidden, drop_prob=0.1):
    super(PositionwiseFeedForward, self).__init__()
    self.linear1=nn.Linear(d_model, hidden)
    self.linear2=nn.Linear(hidden, d_model)
    self.relu=nn.ReLU()
    self.dropout=nn.Dropout(p=drop_prob)

  def forward(self, x):
    x=self.linear1(x)
    x=self.relu(x)
    x=self.dropout(x)
    x=self.linear2(x)
    return x


In [ ]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # 512
        self.beta =  nn.Parameter(torch.zeros(parameters_shape)) # 512

    def forward(self, inputs):
        # inputs : 30 x 200 x 512
        dims = [-(i + 1) for i in range(len(self.parameters_shape))] # [-1]
        #print(f"dims: {dims}")
        mean = inputs.mean(dim=dims, keepdim=True) #30 x 200 x 1
        #print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True) # 30 x 200 x 512
        std = (var + self.eps).sqrt() # 30 x 200 x 512
        #print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std # 30 x 200 x 512
        #print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta  # 30 x 200 x 512
        #print(f"out: {out.size()}")
        return out

In [ ]:
class MultiheadAttention(nn.Module):
  def __init__(self,d_model, num_heads):
    super().__init__()
    #self.input_dim=input_dim
    self.d_model=d_model
    self.num_heads=num_heads
    self.head_dim= d_model // num_heads
    self.qkv_layer= nn.Linear(d_model, 3*d_model)
    self.linear_layer=nn.Linear(d_model, d_model)

  def forward(self, x, mask=None):
    batch_size, seq_length, d_model=x.size()
    qkv=self.qkv_layer(x)
    qkv=qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
    qkv=qkv.permute(0,2,1,3)
    q,k,v=qkv.chunk(3, dim=-1)
    value, attention= scaled_dot_product(q,k,v, mask=None)
    print(f"attention: {attention[0][0]}")
    print(f"attention shape tensor: {attention.shape}")
    #attention=attention.detach().numpy()
    #print(f"after numpy attention shape:{attention.shape}")
    #plt.plot(attention[0][0]) #attention for first word
    #plt.imshow(attention[0][0])
    print(f"values: {value[0][0]}")
    value=value.reshape(batch_size, seq_length, self.num_heads*self.head_dim)
    out=self.linear_layer(value)
    return out


In [ ]:
class MultiheadCrossAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super().__init__()
    self.d_model=d_model
    self.num_heads=num_heads
    self.head_dim= d_model // num_heads
    self.kv_layer=nn.Linear(d_model, 2*d_model)
    self.q_layer= nn.Linear(d_model, d_model)
    self.linear_layer=nn.Linear(d_model, d_model)

  def forward(self,x,y, mask=None):
    batch_size, seq_length, d_model= x.size()
    kv=self.kv_layer(x)
    q=self.q_layer(y)
    kv=kv.reshape(batch_size, seq_length, self.num_heads, 2*self.head_dim)
    q=q.reshape(batch_size, seq_length, self.num_heads, self.head_dim)
    kv=kv.permute(0,2,1,3)
    q=q.permute(0,2,1,3)
    k,v=kv.chunk(2,dim=-1)
    value, attention= scaled_dot_product(q,k,v ,mask)
    value=value.reshape(batch_size, seq_length, d_model)
    out=self.linear_layer(value)
    return out


In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
    super(EncoderLayer, self).__init__()
    self.attention=MultiheadAttention(d_model=d_model, num_heads=num_heads)
    self.norm1=LayerNormalization(parameters_shape=[d_model])
    self.dropout1=nn.Dropout(p=drop_prob)
    self.ffn=PositionwiseFeedForward(d_model, hidden=ffn_hidden, drop_prob=drop_prob)
    self.norm2=LayerNormalization(parameters_shape=[d_model])
    self.dropout2= nn.Dropout(p=drop_prob)

  def forward(self,x ):
    residual_x=x
    x=self.attention(x,mask=None)
    x=self.dropout1(x)
    x=self.norm1(x+residual_x)
    #x=nn.LayerNorm(x+residual_x)
    residual_x=x
    x=self.ffn(x)
    x=self.dropout2(x)
    #x=nn.LayerNorm(x+residual_x)
    x=self.norm2(x+residual_x)
    return x



In [ ]:
class Encoder(nn.Module):
  def __init__(self,d_model, ffn_hidden, num_heads, drop_prop, num_layers):
    super().__init__()
    self.layers=nn.Sequential(*(EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
    for _ in range(num_layers))) # *-> will take a list and deconstruct t in 5 layers

  def forward(self,x):
    x=self.layers(x) #passing through all the encoder layers
    return x


In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
    super(DecoderLayer, self).__init__()
    self.self_attention=MultiheadAttention(d_model=d_model, num_heads=num_heads)
    self.norm1 = LayerNormalization(parameters_shape=[d_model])
    self.dropout1=nn.Dropout(p=drop_prob)
    self.encoder_decoder_attention=MultiheadCrossAttention(d_model, num_heads=num_heads)
    self.norm2 = LayerNormalization(parameters_shape=[d_model])
    self.dropout2=nn.Dropout(p=drop_prob)
    self.ffn=PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
    self.norm3 = LayerNormalization(parameters_shape=[d_model])
    self.dropout3=nn.Dropout(p=drop_prob)

  def forward(self,x, y, decoder_mask):
    _y=y  #residual connections
    #masked self attention
    y=self.self_attention(y, mask=decoder_mask)
    y=self.dropout1(y)
    y=self.norm1(y+_y)
    #y=nn.LayerNorm(y+_y)

    _y=y # cross attention
    y=self.encoder_decoder_attention(x,y, mask=None)
    y=self.dropout2(y)
    y=self.norm2(y+_y)
    #y=nn.LayerNorm(y+_y)

    _y=y #feed forward
    y=self.ffn(y)
    y=self.dropout3(y)
    y=self.norm3(y+_y)
    #y=nn.LayerNorm(y+_y)
    return y

In [ ]:
class SequentialDecoder(nn.Sequential): #because we cant pass more than 1 ip
    def forward(self, *inputs):
        x, y, mask = inputs
        for module in self._modules.values():
            y = module(x, y, mask) #30 x 200 x 512
        return y

In [ ]:
class Decoder(nn.Module):
  def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers=1):
    super().__init__()
    self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
    for _ in range(num_layers)])

  def forward(self, x,y,mask):
    y=self.layers(x,y,mask)
    return y

In [ ]:
class Transformer(nn.Module):
  def __init__(self, d_model, ffn_hidden,num_heads, drop_prob ):
    super().__init__()
    #self.self_attention=MultiheadAttention(d_model=d_model, num_heads=num_heads)
    self.encoder=Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
    self.decoder=Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

  def forward(self, x, y, decoder_mask):
    #x=self.self_attention(x)
    x=self.encoder(x)
    x=self.decoder(x, y, mask)
    return x

In [ ]:
d_model=512
num_heads=8
batch_size=30
drop_prob=0.1
ffn_hidden=2048 #to add non linearity
max_seq_length=200 # max number of words
num_layers= 5 # number of encoders

x=torch.randn( (batch_size, max_seq_length, d_model) ) #input
y=torch.randn( (batch_size, max_seq_length, d_model) ) #ouput
#look ahead mask
mask=torch.full([max_seq_length, max_seq_length], float('-inf'))
mask=torch.triu(mask, diagonal=1)

encoder=Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
out1 = encoder(x)
decoder=Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
out2 =decoder(x,y,mask)

attention: tensor([[0.0032, 0.0045, 0.0052,  ..., 0.0033, 0.0040, 0.0050],
        [0.0064, 0.0038, 0.0060,  ..., 0.0044, 0.0057, 0.0021],
        [0.0042, 0.0043, 0.0083,  ..., 0.0036, 0.0046, 0.0031],
        ...,
        [0.0042, 0.0068, 0.0075,  ..., 0.0043, 0.0063, 0.0049],
        [0.0072, 0.0039, 0.0025,  ..., 0.0033, 0.0042, 0.0066],
        [0.0033, 0.0046, 0.0068,  ..., 0.0034, 0.0055, 0.0056]],
       grad_fn=<SelectBackward0>)
attention shape tensor: torch.Size([30, 8, 200, 200])
values: tensor([[ 0.0618, -0.0036, -0.0387,  ...,  0.0700, -0.0809,  0.0719],
        [ 0.0814, -0.0066, -0.0117,  ...,  0.0872, -0.0855,  0.0614],
        [ 0.0715, -0.0074, -0.0275,  ...,  0.0557, -0.0701,  0.0561],
        ...,
        [ 0.0532,  0.0069, -0.0285,  ...,  0.0807, -0.0692,  0.0312],
        [ 0.0412, -0.0362, -0.0360,  ...,  0.0591, -0.0933,  0.0572],
        [ 0.0709, -0.0192, -0.0076,  ...,  0.0358, -0.0726,  0.0461]],
       grad_fn=<SelectBackward0>)
attention: tensor([[0.0059,

In [ ]:
transformer= Transformer(d_model, ffn_hidden, num_heads, drop_prob)
out3=transformer(x, y, mask)

attention: tensor([[0.0074, 0.0101, 0.0035,  ..., 0.0061, 0.0070, 0.0046],
        [0.0056, 0.0073, 0.0058,  ..., 0.0034, 0.0031, 0.0054],
        [0.0024, 0.0078, 0.0061,  ..., 0.0036, 0.0035, 0.0064],
        ...,
        [0.0034, 0.0060, 0.0054,  ..., 0.0058, 0.0028, 0.0034],
        [0.0066, 0.0090, 0.0055,  ..., 0.0042, 0.0047, 0.0043],
        [0.0077, 0.0053, 0.0030,  ..., 0.0058, 0.0036, 0.0032]],
       grad_fn=<SelectBackward0>)
attention shape tensor: torch.Size([30, 8, 200, 200])
values: tensor([[-0.0255, -0.0141, -0.0201,  ...,  0.0431, -0.0645,  0.0042],
        [ 0.0054, -0.0365,  0.0092,  ...,  0.0208, -0.0529,  0.0346],
        [ 0.0262, -0.0643,  0.0592,  ...,  0.0459, -0.0875,  0.0373],
        ...,
        [ 0.0196, -0.0524,  0.0473,  ...,  0.0376, -0.0476,  0.0319],
        [ 0.0013, -0.0361,  0.0178,  ...,  0.0379, -0.0783,  0.0283],
        [-0.0079, -0.0458,  0.0225,  ...,  0.0165, -0.0635, -0.0593]],
       grad_fn=<SelectBackward0>)
attention: tensor([[0.0033,